In [ ]:
!pip install mmcv-full==1.3.8 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -e .

In [ ]:
import sys
sys.path.insert(0, "./mmdetection")

import mmdet
import mmdet.core
from mmdet.apis import init_detector, inference_detector
import mmcv

In [ ]:
from mmdet.core.bbox import demodata

In [ ]:
import pandas as pd
import os

In [ ]:
level_df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
level_df

In [ ]:
meta = pd.read_csv('../input/second/metadf.csv')
meta["id"] += "_image"
meta

In [ ]:
merge_df = pd.merge(level_df, meta, on="id", how="left")
merge_df

In [ ]:
!conda install gdcm -c conda-forge -y

In [ ]:
from tqdm.auto import tqdm


In [ ]:
anno_df = pd.DataFrame(columns = [[ 'image_id','bbox_name', 'bbox']])
anno_df

In [ ]:
bbox_names = []
bboxes = []

image_len = level_df.shape[0]

for i, row in tqdm(merge_df.iterrows(), total=merge_df.shape[0]):
    image_id = row.id.split('_')[0]
    boxes = row.label.split()
    bbox_name = []
    bbox = []
    sum = []
    width_ratio = 512/row.dim1
    height_ratio = 512/row.dim0
    if boxes[0] == 'none':
        anno_df.loc[i] = [image_id, None, None]
        continue
    else:
        for j in range(int(len(boxes)/6)):
            label = boxes[j*6+0]
            x1 = float(boxes[j*6+2])*width_ratio
            y1 = float(boxes[j*6+3])*height_ratio
            x2 = float(boxes[j*6+4])*width_ratio
            y2 = float(boxes[j*6+5])*height_ratio

            bbox_name.append(label)
            bbox.append([float(x1), float(y1), float(x2), float(y2)])
    sum.append(image_id)
    sum.append(bbox_name)
    sum.append(bbox)
    anno_df.loc[i] = sum
anno_df

In [ ]:
anno_df = anno_df.dropna().reset_index(drop=True)
print(anno_df.shape)

In [ ]:
anno_df

In [ ]:
merge_df = merge_df.dropna().reset_index(drop=True)
print(merge_df.shape)

In [ ]:
merge_df

In [ ]:
# merge_df.loc[0,"boxes"]

In [ ]:
# merge_df.loc[0,"label"]

In [ ]:
# anno_df.loc[0,"bbox"][0]

In [ ]:
# aaaa = anno_df.values.tolist()
# aaaa[0][2]


In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# image = cv2.imread('/kaggle/working/siim/image/000a312787f2_image.png')
# draw_img = image.copy()
# cv2.rectangle(draw_img, (94,85) , (218, 366), (0,255,0), 2)
# cv2.rectangle(draw_img, (270,86) , (401, 345), (0,255,0), 2)
# # cv2.putText(draw_img, caption, (94.95198315789473, 85.49436330275229-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, red_color, 2)
# plt.imshow(draw_img)


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(merge_df, test_size=0.1,random_state=2021)

In [ ]:
print(len(train_df), len(val_df))


In [ ]:
import os
os.mkdir('/kaggle/working/siim')

In [ ]:
train_df['id'].to_csv('/kaggle/working/siim/train.txt', sep=' ', header=False, index=False)
val_df['id'].to_csv('/kaggle/working/siim/val.txt', sep=' ', header=False, index=False)

In [ ]:
os.mkdir('/kaggle/working/siim/image')

In [ ]:
import shutil
for i in range(len(merge_df)):
    shutil.copy(f'../input/second/image/{merge_df.loc[i,"id"]}.png', "/kaggle/working/siim/image") 

In [ ]:
ls /kaggle/working/siim

In [ ]:

# os.remove('/kaggle/working/siim/image')
# shutil.rmtree('/kaggle/working/siim/image')

In [ ]:

# image = cv2.imread('/kaggle/working/siim/image/000a312787f2_image.png')
# height, width = image.shape[:2]
# print(height, width)

In [ ]:
# image_list = mmcv.list_from_file('/kaggle/working/siim/train.txt')
# image_list

In [ ]:
import copy
import os.path as osp

import mmcv
import numpy as np
import cv2

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset


@DATASETS.register_module(force=True)
class CovDataset(CustomDataset):
    CLASSES = ['opacity']

    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)

        data_infos = []

        for image_id in image_list:

            filename = '{0:}/{1:}.png'.format(self.img_prefix, image_id)
            image = cv2.imread(filename)
            height, width = image.shape[:2]

            data_info = {'filename': str(image_id)+'.png',
                      'width': width, 'height': height}
            df_list = anno_df.values.tolist()
            bbox_names = df_list[i][1]
            bboxes = df_list[i][2]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []


            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)


            data_anno = {
            'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
            'labels': np.array(gt_labels, dtype=np.long),
            'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
            'labels_ignore': np.array(gt_labels_ignore, dtype=np.long)
            }


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos


In [ ]:
ls /kaggle/working/mmdetection/

In [ ]:
config_file = "/kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py"
# checkpoint_file = '/kaggle/working/mmdetection/checkpoints/cascade_rcnn_x101_64x4d_fpn_1x_coco_20200515_075702-43ce6a30.pth'

In [ ]:
# !cd mmdetection; mkdir checkpoints
# !wget -O /kaggle/working/mmdetection/checkpoints/cascade_rcnn_x101_64x4d_fpn_1x_coco_20200515_075702-43ce6a30.pth \
# https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco/cascade_rcnn_x101_64x4d_fpn_1x_coco_20200515_075702-43ce6a30.pth

In [ ]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

In [ ]:
import os

job_folder = '/kaggle/working/siim_work_dir'

os.mkdir(job_folder)

print("job_folder:", job_folder)
# !mkdir /kaggle/working/siim_work_dir

In [ ]:
from mmdet.apis import set_random_seed

cfg.dataset_type = 'CovDataset'
cfg.data_root = '/kaggle/working/siim/'

cfg.data.train.type = 'CovDataset'
cfg.data.train.data_root = '/kaggle/working/siim/'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'image'

cfg.data.val.type = 'CovDataset'
cfg.data.val.data_root = '/kaggle/working/siim/'
cfg.data.val.ann_file = 'val.txt'         
cfg.data.val.img_prefix = 'image'

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1
# cfg.model.roi_head.bbox_head[0]["num_classes"] = 1
# cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

cfg.work_dir = job_folder

cfg.optimizer.lr = 0.02 / 8 
cfg.lr_config.warmup = None #None => linear
cfg.log_config.interval = 5

cfg.runner.max_epochs = 10


cfg.evaluation.metric = 'mAP'
cfg.evaluation.interval = 5
cfg.checkpoint_config.interval = 5

cfg.data.samples_per_gpu = 4
# cfg.evaluation.iou_thrs = [0.5]

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.lr_config.policy='step' #step => CosineAnnealing

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성. 

datasets = [build_dataset(cfg.data.train)]

In [ ]:
datasets

In [ ]:
# !pwd

In [ ]:
%cd mmdetection

model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
# !ls /kaggle/working/siim_work_dir

In [ ]:
# %cd

In [ ]:
# from mmdet.apis import show_result_pyplot
# import matplotlib.pyplot as plt
# checkpoint_file = '/kaggle/working/siim_work_dir/epoch_20.pth'

# # checkpoint 저장된 model 파일을 이용하여 모델을 생성, 이때 Config는 위에서 update된 config 사용. 
# model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')
# # BGR Image 사용 
# img = cv2.imread('/kaggle/working/siim/image/b8b0eda83a15_image.png')
# # model_ckpt.cfg = cfg
# print(img.shape)

In [ ]:
# result = inference_detector(model_ckpt,img)
# result



In [ ]:
# show_result_pyplot(model_ckpt, img, result, score_thr=0.2)

In [ ]:
# !echo 'val list #####'; cat /kaggle/working/siim/val.txt